# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121428e+02     1.632101e+00
 * time: 0.09272313117980957
     1     1.078605e+01     8.844887e-01
 * time: 0.9745299816131592
     2    -1.175492e+01     1.051550e+00
 * time: 1.101747989654541
     3    -3.415221e+01     7.962550e-01
 * time: 1.2628769874572754
     4    -4.768100e+01     5.502533e-01
 * time: 1.4153931140899658
     5    -5.685251e+01     1.829604e-01
 * time: 1.562648057937622
     6    -5.972192e+01     1.124940e-01
 * time: 1.6787149906158447
     7    -6.083784e+01     5.240263e-02
 * time: 1.7876172065734863
     8    -6.128258e+01     6.077439e-02
 * time: 1.9051179885864258
     9    -6.159752e+01     3.548038e-02
 * time: 2.0152080059051514
    10    -6.182750e+01     2.819221e-02
 * time: 2.133674144744873
    11    -6.199413e+01     2.220244e-02
 * time: 2.2517571449279785
    12    -6.206273e+01     1.720743e-02
 * time: 2.3619771003723145
    13    -6.213211e+01     1.427430e-02
 * time: 2.481270074844

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671060
    AtomicLocal         -18.8557640
    AtomicNonlocal      14.8522623
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485358 
    Xc                  -19.3336815

    total               -62.261666452935
